# redundant and hacky implementation of BFS just to get some sample data

In [1]:
from selenium import webdriver
import pandas as pd


def validate_url(driver, url):
    ''' if valid url, return set (ie unique) of links otherwize return None'''
    try:
        driver.get(url)
        ids =  driver.find_elements_by_xpath('//*[@href]')
        return set([x.get_attribute('href') for x in ids])
    except:
        return set()

    
def bfs(seed_url, driver, frames, limit):
    ''' get all links from a given url'''
    # set params for DF organization
    depth = 0
    node_id = 0
    parent = 0
    # dont repeat (endless cycles)
    visited_set = set([seed_url])
    #initilize seed row
    frames[depth] = pd.DataFrame(dict(depth=[depth], node_id=[node_id], parent=[parent], url=[seed_url]))
    depth += 1
    # get links for seed
    links = validate_url(driver, seed_url)
    # remove ones that have been seen
    links = links - visited_set
    # used for unique identier of node for graphing
    new_node_id = len(links) + node_id + 1
    # build dataframe and append to list of frames
    temp_frame = pd.DataFrame(dict(node_id=list(range(node_id + 1,new_node_id)), url=list(links)))
    temp_frame['depth'] = depth
    temp_frame['parent'] = parent
    frames[depth] = (temp_frame)
    #update node id
    node_id = new_node_id - 1
    # update visited set
    visited_set.update(links)

    for i in range(depth, limit):
        depth += 1
        temp_level = list()
        for _,row in frames[i].iterrows():
            links = validate_url(driver, row['url'])
            links = links - visited_set
            if len(links) > 0:
                new_node_id = len(links) + node_id + 1
                temp_frame = pd.DataFrame(dict(node_id=list(range(node_id + 1, new_node_id)), url=list(links)))
                temp_frame['depth'] = depth
                temp_frame['parent'] = row['node_id']
                temp_level.append(temp_frame)
                node_id = new_node_id - 1
                visited_set.update(links)
            
        frames[depth] = pd.concat(temp_level)
            
    return pd.concat(frames.values())

In [2]:
# look at data from my website (not many links and still graph grows huge! )

In [3]:
driver = webdriver.Firefox()
driver.set_page_load_timeout(5)
frames = dict()
tr = bfs("https://www.cas-donoghue.org/", driver, frames, 2)

driver.close()
tr.to_csv("look.csv")
tr.head()

,depth,node_id,parent,url
0,0,0,0,https://www.cas-donoghue.org/
0,1,1,0,https://twitter.com/mkennedy
1,1,2,0,https://www.cas-donoghue.org/home
2,1,3,0,https://talkpython.fm/
3,1,4,0,https://www.linkedin.com/in/casadilla


In [4]:
tr.head(50)

,depth,node_id,parent,url
0,0,0,0,https://www.cas-donoghue.org/
0,1,1,0,https://twitter.com/mkennedy
1,1,2,0,https://www.cas-donoghue.org/home
2,1,3,0,https://talkpython.fm/
3,1,4,0,https://www.linkedin.com/in/casadilla
4,1,5,0,https://www.cas-donoghue.org/static/css/landin...
5,1,6,0,https://www.cas-donoghue.org/static/img/code_o...
6,1,7,0,https://stackoverflow.com/users/9034974/cas-do...
7,1,8,0,https://www.cas-donoghue.org/static/css/theme....
8,1,9,0,https://www.cas-donoghue.org/static/bower_comp...


# build frame for visualization

In [6]:
def make_x_list(val):
    '''spread x coord around 0'''
    x = int(val/2)
    if val%2 != 0:
        return list(range(x*-1, x + 1))
    else:
        return list(range(x*-1, x))
    
tr['node_y'] = tr['depth'] * -1
count_series = tr['depth'].value_counts()
index_dict = dict()
y_col = list()

for i,v in count_series.iteritems():
    index_dict[i] = make_x_list(v)

for key in sorted(index_dict.keys()):
    y_col.extend(index_dict[key])
tr['node_x'] = y_col


    
tr.head()

,depth,node_id,parent,url,node_y,node_x
0,0,0,0,https://www.cas-donoghue.org/,0,0
0,1,1,0,https://twitter.com/mkennedy,-1,-9
1,1,2,0,https://www.cas-donoghue.org/home,-1,-8
2,1,3,0,https://talkpython.fm/,-1,-7
3,1,4,0,https://www.linkedin.com/in/casadilla,-1,-6


In [7]:
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.io import output_notebook
output_notebook()
source = ColumnDataSource(data=dict(node_x=tr['node_x'], node_y=tr['node_y'], desc=tr['url']))

hover = HoverTool(tooltips=[('URL', "@desc")])

p = figure(plot_width=900, plot_height=400, title="BFS prototype")
p.circle('node_x', 'node_y', size=5, source=source)
p.multi_line([[-9,0], [-8,0]],[[-1,0], [-1,0]], color=['blue', 'red'], line_width=1)
p.add_tools(hover)
show(p)

Loading BokehJS ...

In [ ]:
527/2